In [1]:
import glob 

import math
from datetime import timedelta
from operator import attrgetter

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize 
import cartopy.crs as ccrs  

import parcels

In [2]:
def WrapLongitudeKernel(particle, fieldset, time):
    if particle.lon > 180:
        particle.dlon = -360
    elif particle.lon < -180:
        particle.dlon = 360

def ConvertParticlesTo360(particle, fieldset, time):
    if particle.lon < 0:
        particle.lon += 360  # Shift to 0-360 range

def ConvertParticlesTo180(particle, fieldset, time):
    if particle.lon > 180:
        particle.lon -= 360  # Shift back to -180,180 range

def DeleteParticle(particle, fieldset, time):
    particle.delete()  # Remove particle from the simulation

class DrifterParticle(parcels.JITParticle):
    age = parcels.Variable("age", dtype=np.float32, initial=0, to_write=True)  
    drifter_id = parcels.Variable("drifter_id", dtype=np.float32, to_write=True) 

def Age(particle, fieldset, time):
    particle.age += particle.dt / 3600 # in hours

def run_parcels_test(model: str, filenames: dict, variables: dict, dimensions: dict, indices: dict, drifter_df: pd.DataFrame, T: int, dt: int, savet: int, ):
    
    # QC 
    print("QC:")

    if model == "AMPHITRITE":
        model_xr = xr.open_dataset(filenames.get("U")[0]) 
    else: 
        model_xr = xr.open_dataset(glob.glob(filenames.get("U"))[0]) 
 
    for attr in model_xr[variables.get("U")].attrs:
        print(attr, ":", model_xr[variables.get("U")].attrs[attr])

    for attr in model_xr.attrs:
        print(attr, ":", model_xr.attrs[attr])

    if model == "ROMS":
        fieldset = parcels.FieldSet.from_mitgcm(filenames, variables, dimensions, indices, allow_time_extrapolation=True)
    else: 
        fieldset = parcels.FieldSet.from_netcdf(filenames, variables, dimensions, indices, allow_time_extrapolation=True)

    print("")
    print("Dataset size:", len(fieldset.U.grid.lon)*len(fieldset.U.grid.lat)*len(fieldset.U.grid.time)) 
    print("Domain:", fieldset.U.grid.lon.min(), fieldset.U.grid.lon.max(),  fieldset.U.grid.lat.min(), fieldset.U.grid.lat.max())
    print("")

    # Define a new particleclass with Variable 'age' with initial value 0.
    # AgeParticle = parcels.JITParticle.add_variable(parcels.Variable("age", initial=0))

    pset = parcels.ParticleSet.from_list(
        fieldset=fieldset,
        pclass=DrifterParticle,
        lon=drifter_df["lon"].values,
        lat=drifter_df["lat"].values,
        time=drifter_df["time"].values,
        drifter_id=drifter_df["id_nr"].values.astype(np.float32)
    )

    output_file = pset.ParticleFile(
        name=f"{model}Particles.zarr", outputdt=timedelta(hours=savet)
    )

    if model =="HYCOM":
        kernels = [Age, ConvertParticlesTo360, parcels.AdvectionRK4]

        pset.execute(
            kernels, # pset.Kernel(ConvertParticlesTo360) + parcels.AdvectionRK4,
            runtime=timedelta(hours=T),
            dt=timedelta(hours=dt),
            output_file=output_file
        )

        # Convert back before saving output
        #pset.execute(
            # pset.Kernel(ConvertParticlesTo180),
            # runtime=timedelta(seconds=0),  # Just apply conversion
            # dt=timedelta(seconds=0)
        # )

        for particle in pset:
            if particle.lon > 180:
                particle.lon -= 360  # Convert from 0-360 to -180,180
    
    else:
        kernels = [Age, parcels.AdvectionRK4]

        pset.execute(
            kernels, # parcels.AdvectionRK4,
            runtime=timedelta(hours=T),
            dt=timedelta(hours=dt),
            output_file=output_file
        )

    return

In [ ]:
"""
ics_df = tracks_df.set_index("time")
non_numerical_cols = ics_df.select_dtypes(exclude="number")

filled_non_numerical = (
    non_numerical_cols.groupby("id")
    .apply(lambda group: group.resample("1D").ffill())
)
ics_df = ics_df.groupby("id").resample("1D").interpolate().drop(columns=["id","drogue"]).reset_index()
ics_df["segment_id"] = ics_df["segment_id"].astype(int)
ics_df = ics_df.merge(filled_non_numerical.drop(columns=["id"]).reset_index(), how="left")
"""

# WP 1C - Setup

#### Load drifter tracks

In [ ]:
# Load drifter tracks 
drifter_folder = r"Y:/PROJECTS/DRIFTERS/data/qc_tdsi_6h_2"
files = glob.glob(f"{drifter_folder}/*.csv")

drifter_df = pd.DataFrame()
for file in files: 
    df = pd.read_csv(file)  
    drifter_df = pd.concat([drifter_df, df], ignore_index=True)  

drifter_df.rename(columns={"longitude": "lon", "latitude": "lat"}, inplace=True)
drifter_df = drifter_df[drifter_df["deployed"] != False]
drifter_df.drop(columns="deployed", inplace=True)

In [ ]:
w, e, s, n = drifter_df.lon.min(), drifter_df.lon.max(), drifter_df.lat.min(), drifter_df.lat.max()
w, e, s, n = int(w), int(e), int(s), int(n)

tstart, tend = drifter_df.time.min(), drifter_df.time.max()

d_nr = drifter_df.id.count()

print("Number of drifters:", d_nr)
print("Time range:", tstart, tend)
print("W, E, S, N:", [w, e, s, n])

In [107]:
w, e, s, n = -160, -125, 20, 40
tstart = '2022-01-01'
tend = '2022-01-31'

In [ ]:
# Filter 
wesn = (drifter_df.lon > w) & (drifter_df.lon < e) & (drifter_df.lat > s) & (drifter_df.lat < n)
time = (drifter_df.time >= tstart) & (drifter_df.time <= tend)
drifter_df["time"] = pd.to_datetime(drifter_df["time"])
tracks_df = drifter_df[time & wesn].reset_index(drop=True)
tracks_df["id_nr"] = tracks_df.groupby("id").ngroup()

ics_df = tracks_df.groupby("id_nr", group_keys=False).apply(lambda x: x.iloc[::4])

In [ ]:
w, e, s, n = ics_df.lon.min(), ics_df.lon.max(), ics_df.lat.min(), ics_df.lat.max()
w, e, s, n = int(w), int(e), int(s), int(n)

tstart, tend = ics_df.time.min(), ics_df.time.max()

d_nr = ics_df.id_nr.max()

print("Time range: ", tstart, tend)
print("W, E, S, N:", [w, e, s, n])
print("Number of drifters:", d_nr)

In [125]:
# ics_df = ics_df.groupby("id").first()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

scatter = ax.scatter(ics_df.lon, ics_df.lat, s=10, c=ics_df["id_nr"], cmap="tab20c", edgecolor="k")

cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('Drifter ID', fontsize=12)

ax.set_title(f"Map of Initial Conditions ({ics_df.time.iloc[0].strftime('%Y-%m-%d')} - {ics_df.time.iloc[-1].strftime('%Y-%m-%d')})", fontsize=16)
ax.gridlines(draw_labels=True)
ax.coastlines(resolution='110m', color='black', linewidth=1)
ax.set_xlabel("Longitude", fontsize=14)
ax.set_ylabel("Latitude", fontsize=14)
ax.set_extent([ics_df.lon.min(), ics_df.lon.max(), ics_df.lat.min(), ics_df.lat.max()], crs=ccrs.PlateCarree())

plt.tight_layout()
plt.show()

#### Load OGCMs

Amphitrite training period: 2018-2021 and 08/2023-02/2024 

#### Resources <br>
Grid:  <br>
https://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/build/ch05s06.html <br> 
https://www.oc.nps.edu/nom/modeling/grids.html <br> 
<br> 
Conventions: <br> 
http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html

In [114]:
T = int((tracks_df.time.max() - tracks_df.time.min()).total_seconds()/3600) # 7*24 
dt = 1
savet = 6 

#### GLOBCURRENT

In [126]:
dataset_folder = "F:/ADVECTOR/metocean"

In [127]:
model = "GLOBCURRENT"

filenames = {
    "U": f"{dataset_folder}/globcurrent/uo_*.nc",
    "V": f"{dataset_folder}/globcurrent/vo_*.nc"
}
variables = {
    "U": "uo",
    "V": "vo",
}

dimensions = {"time": "time", "lat": "latitude", "lon": "longitude", "time": "time"}

indices = {'lon': range(0,4*60), 'lat': range(4*105,4*135)}

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

In [ ]:
# FIXME: ChatGPT 

# Load dataset
ds = xr.open_zarr("GLOBCURRENTParticles.zarr") 
tracks = tracks_df[tracks_df.id_nr < 10]

# Mask dataset for selected drifters
ds = ds.where(ds.drifter_id.isin(tracks.id_nr.unique()).compute(), drop=True)
drifter_ids = tracks.id_nr.unique()

# Create figure
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Define colormap
cmap_id = plt.get_cmap("tab20c", len(drifter_ids))  # Unique color per drifter ID
norm = Normalize(vmin=drifter_ids.min(), vmax=drifter_ids.max())  # Normalize ID values for color mapping

# Scatter plot for each drifter
sc = None  # Placeholder for colorbar reference
for i, (id, drifter) in enumerate(tracks.groupby("id_nr")): 
    track = drifter  # No need to redefine it with filtering again
    sc = ax.scatter(track.lon, track.lat, c=[id] * len(track), cmap=cmap_id, norm=norm, s=50, edgecolor="k", marker="v", zorder=0)

    # Filter dataset for particle positions
    mask = ds.drifter_id == id
    age_mask = (ds.drifter_id == id) & (ds.age == 1)

    if len(ds.lon.values[np.where(mask.values)]) == 0:
        print(f"Missing particle for ID {id}")

    ax.scatter(ds.lon.values[age_mask], ds.lat.values[age_mask], color=cmap_id(i), s=50, marker="s", zorder=2)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('Drifter ID', fontsize=12)

# Labels and styling
ax.set_title(f"Sample of Drifter Tracks vs Initial Conditions ({tracks_df.time.iloc[0].strftime('%Y-%m-%d')} - {tracks_df.time.iloc[-1].strftime('%Y-%m-%d')})", fontsize=16)
ax.gridlines(draw_labels=True)
ax.coastlines(resolution='110m', color='black', linewidth=1)
ax.set_xlabel("Longitude", fontsize=14)
ax.set_ylabel("Latitude", fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
ds = xr.open_zarr("GLOBCURRENTParticles.zarr") 

tracks = tracks_df[tracks_df.id_nr < 10]

ds = xr.open_zarr(f"GLOBCURRENTParticles.zarr")
drifter_mask = ds.drifter_id.isin(tracks.id_nr.unique()).compute()
ds = ds.where(drifter_mask, drop=True)

drifter_ids = tracks.id_nr.unique() # np.unique(ds.drifter_id.values)  

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})
cmap_id = plt.get_cmap("tab20c", len(drifter_ids))  # Unique color per drifter ID
cmap_age = plt.get_cmap("viridis")  # Age gradient

for i, (id, drifter) in enumerate(tracks.groupby("id_nr")): 
    # print(f"{i}/{len(drifter_ids)}")

    track = tracks[tracks["id_nr"] == id]
    ax.scatter(track.lon, track.lat, color=cmap_id(i), s= 50, edgecolor="k", marker="v", zorder = 0)
        
    drifter_mask = ds.drifter_id == id 
    age_mask = (ds.drifter_id == id) & (ds.age == 1)
    mask_indices = np.where(drifter_mask.values)

    if len(ds.lon.values[mask_indices]) == 0:
        print("Missing particle")

    # ax.scatter(ds.lon.values[mask_indices], ds.lat.values[mask_indices], color=cmap_id(i), alpha=0.5, s= ds.age.values[mask_indices]/10, zorder = 1)
    ax.scatter(ds.lon.values[age_mask], ds.lat.values[age_mask], color=cmap_id(i), s= 50, marker = "s", zorder = 2)

# cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)
# cbar.set_label('Drifter ID', fontsize=12)

ax.set_title(f"Sample of Drifter Tracks vs Initial Conditions ({ics_df.time.iloc[0].strftime('%Y-%m-%d')} - {ics_df.time.iloc[-1].strftime('%Y-%m-%d')})", fontsize=16)
ax.gridlines(draw_labels=True)
ax.coastlines(resolution='110m', color='black', linewidth=1)
ax.set_xlabel("Longitude", fontsize=14)
ax.set_ylabel("Latitude", fontsize=14)

plt.tight_layout()
plt.show()

### GLORYS

In [323]:
model = "GLORYS"

filenames = {
    "U": f"{dataset_folder}/nemo/nemo_hourly_operational_u_*.nc",
    "V": f"{dataset_folder}/nemo/nemo_hourly_operational_v_*.nc",
}

variables = {"U": "uo", "V": "vo"}

dimensions = {"time": "time", "lon": "longitude", "lat": "latitude"}

indices = {"lon": range(0,12*60), "lat": range(12*95,12*125)}

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

### SMOC

In [325]:
model = "SMOC"

filenames = {
    "U": f"{dataset_folder}/nemo_tot/nemo_hourly_operational_utot_*.nc",
    "V": f"{dataset_folder}/nemo_tot/nemo_hourly_operational_vtot_*.nc",
}

variables = {"U": "utotal", "V": "vtotal"}

dimensions = {"time": "time", "lon": "longitude", "lat": "latitude"}

indices = {"lon": range(0,12*60), "lat": range(12*95,12*125)}

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

### HYCOM

In [ ]:
"""
for file in glob.glob(f"{dataset_folder}/hycom/hycom_operational_*.nc"):
    ds = xr.open_dataset(file)
    ds["lon"] = ((ds["lon"] + 180) % 360) - 180
    ds.attrs["geospatial_lon_min"] = -180.0  # Convert from 0.0
    ds.attrs["geospatial_lon_max"] = 179.92  # Convert from 359.92
    ds = ds.sortby("lon")
    ds.to_netcdf(file)
"""

In [327]:
model = "HYCOM"

filenames = {"U": f"{dataset_folder}/hycom/hycom_operational_*.nc",
             "V": f"{dataset_folder}/hycom/hycom_operational_*.nc",
            }

variables = {"U": "water_u", "V": "water_v"}

dimensions = {"time": "time", "lon": "lon", "lat": "lat"}

indices = {'lon': range(int(180*12.5), int(240*12.5)), 'lat': range(int(25*95),int(25*125))}

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

### ROMS

In [ ]:
"""
roms = xr.open_dataset(glob.glob(f"{dataset_folder}/roms/hycom_20220101.nc")[0])
roms.isel(time=0, depth=0).u.plot(vmin=-1)
roms = xr.open_dataset(glob.glob(f"{dataset_folder}/roms/nemo_20220101.nc")[0])
roms.isel(time=0, depth=0).u.plot(vmin=-1)

print(np.shape(roms.u))
print(np.shape(roms.v))
print(np.shape(roms.h))

nind = 3
plt.scatter(roms.lon_u.values[:nind, :nind], roms.lat_u.values[:nind, :nind], c ="r", label = "U grid")
plt.scatter(roms.lon_v.values[:nind, :nind], roms.lat_v.values[:nind, :nind], c ="b", label = "V grid")
plt.scatter(roms.lon_rho.values[:nind, :nind], roms.lat_rho.values[:nind, :nind], c ="k", label = "C (rho) grid")
plt.legend(loc="upper right")
"""

In [338]:
model = "ROMS"

filenames = {"U": f"{dataset_folder}/roms/test134cars_npo0.08_07e_2022*.nc", 
             "V": f"{dataset_folder}/roms/test134cars_npo0.08_07e_2022*.nc"
}

variables = {"U": "u", "V": "v"}

# Note that all variables need the same dimensions in a C-Grid
c_grid_dimensions = {
    "lon": "lon_rho",
    "lat": "lat_rho",
    "time": "ocean_time",
}

dimensions = {
    "U": c_grid_dimensions,
    "V": c_grid_dimensions,
}

indices = {"lon": range(0,600), "lat": range(0,375)}

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

### Amphitrite

In [336]:
model = "AMPHITRITE"

paths = []

folders = glob.glob(f"{dataset_folder}/amphitrite/*")
for folder in folders: 
    paths.append(glob.glob(f"{folder}/*.nc")[0])

filenames = {"U": paths, 
             "V": paths
}

variables = {"U": "u", "V": "v"}

dimensions = {"time": "time", "lon": "longitude", "lat": "latitude"}

indices = None

In [ ]:
run_parcels_test(model, filenames, variables, dimensions, indices, ics_df, T, dt, savet)

### Analysis

In [ ]:
# models = ["GLOBCURRENT", "GLORYS", "SMOC", "HYCOM", "ROMS", "AMPHITRITE"]
# colors = ["purple", "red", "orange", "blue", "cyan", "green"]

models = ["GLOBCURRENT", "GLORYS", "SMOC", "HYCOM", "AMPHITRITE"]
colors = ["purple", "red", "orange", "blue", "green"]

# Create a map with Cartopy GeoAxes
fig, axs = plt.subplots(2,1, figsize=(20, 10), subplot_kw={'projection': ccrs.PlateCarree()})

for ax in axs:
    # Plot the trajectories for each model
    for k, model in zip(colors, models):
        ds = xr.open_zarr(f"{model}Particles.zarr")
        for 
        ds.traj.plot(ax=ax, label=model, color=k, alpha=0.7)

    # Plot drifter samples (scatter plot for each drifter group)
    # Track whether the drifter label has been used already
    label_used = False
    for id, drifter in tracks_df.groupby("id"): 
        ax.plot(drifter.lon, drifter.lat, 'k', linewidth=2, label="Drifter Sample" if not label_used else "")
        label_used = True  # Ensure the label is used only once

    # Add labels and title
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("Particle Trajectories and Drifter Samples (2022-01-01 to 2022-01-07)")
    ax.grid(True)

    # Improve the legend (remove duplicate labels)
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))

axs[0].legend(by_label.values(), by_label.keys())

axs[1].set_xlim(-140,-145)
axs[1].set_ylim(30,35)

plt.show()

#### Advection

In [ ]:
# parcels.timer.root = parcels.timer.Timer("root")##
# parcels.timer.fieldset = parcels.timer.Timer("fieldset creation", parent=parcels.timer.root)
# parcels.timer.fieldset.stop()
# # parcels.timer.root.stop()
# parcels.timer.root.print_tree()

In [ ]:
ds = xr.open_zarr("XXXParticles.zarr")
ds.traj.plot(margin=2)
plt.show()

## Sampling a Field with Particles

https://archimer.ifremer.fr/doc/00157/26792/24888.pdf 

In [29]:
# this flow does not depend on time, we need to set allow_time_extrapolation=True when reading in the fieldset

example_dataset_folder = parcels.download_example_dataset("Peninsula_data")
fieldset = parcels.FieldSet.from_parcels(
    f"{example_dataset_folder}/peninsula",
    extra_fields={"P": "P"},
    allow_time_extrapolation=True,
)

In [30]:
SampleParticle = parcels.JITParticle.add_variable("p")

In [ ]:
pset = parcels.ParticleSet.from_line(
    fieldset=fieldset,
    pclass=SampleParticle,
    start=(3000, 3000),
    finish=(3000, 46000),
    size=5,
    time=0,
)

plt.contourf(fieldset.P.grid.lon, fieldset.P.grid.lat, fieldset.P.data[0, :, :])
plt.xlabel("Zonal distance [m]")
plt.ylabel("Meridional distance [m]")
plt.colorbar()

plt.plot(pset.lon, pset.lat, "ko")
plt.show()

In [34]:
def SampleP(particle, fieldset, time):
    """Custom function that samples fieldset.P at particle location"""
    particle.p = fieldset.P[time, particle.depth, particle.lat, particle.lon]

In [ ]:
output_file = pset.ParticleFile(
    name="PeninsulaPressure.zarr", outputdt=timedelta(hours=1)
)
pset.execute(
    [parcels.AdvectionRK4, SampleP],  # list of kernels to be executed
    runtime=timedelta(hours=20),
    dt=timedelta(minutes=5),
    output_file=output_file,
)

In [ ]:
ds = xr.open_zarr("PeninsulaPressure.zarr")

plt.contourf(fieldset.P.grid.lon, fieldset.P.grid.lat, fieldset.P.data[0, :, :])
plt.xlabel("Zonal distance [m]")
plt.ylabel("Meridional distance [m]")
plt.colorbar()

plt.scatter(ds.lon, ds.lat, c=ds.p, s=30, cmap="viridis", edgecolors="k")
plt.show()

## Calculating distance travelled

In [37]:
# to_write to write in output file

extra_vars = [
    parcels.Variable("distance", initial=0.0, dtype=np.float32),
    parcels.Variable(
        "prev_lon", dtype=np.float32, to_write=False, initial=attrgetter("lon")
    ),
    parcels.Variable(
        "prev_lat", dtype=np.float32, to_write=False, initial=attrgetter("lat")
    ),
]

DistParticle = parcels.JITParticle.add_variables(extra_vars)

In [38]:
def TotalDistance(particle, fieldset, time):
    """Calculate the distance in latitudinal direction
    (using 1.11e2 kilometer per degree latitude)"""
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    lon_dist = (
        (particle.lon - particle.prev_lon)
        * 1.11e2
        * math.cos(particle.lat * math.pi / 180)
    )
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    # Set the stored values for next iteration
    particle.prev_lon = particle.lon
    particle.prev_lat = particle.lat

In [39]:
example_dataset_folder = parcels.download_example_dataset("GlobCurrent_example_data")
filenames = {
    "U": f"{example_dataset_folder}/20*.nc",
    "V": f"{example_dataset_folder}/20*.nc",
}
variables = {
    "U": "eastward_eulerian_current_velocity",
    "V": "northward_eulerian_current_velocity",
}
dimensions = {"lat": "lat", "lon": "lon", "time": "time"}
fieldset = parcels.FieldSet.from_netcdf(filenames, variables, dimensions)
pset = parcels.ParticleSet.from_line(
    fieldset=fieldset, pclass=DistParticle, size=5, start=(28, -33), finish=(30, -33)
)

In [ ]:
pset.execute(
    [parcels.AdvectionRK4, TotalDistance],  # list of kernels to be executed
    runtime=timedelta(days=6),
    dt=timedelta(minutes=5),
    output_file=pset.ParticleFile(
        name="GlobCurrentParticles_Dist.zarr", outputdt=timedelta(hours=1)
    ),
)

In [ ]:
print([p.distance for p in pset])